# **Rock-Paper-Scissors Classification with Convolutional Neural Networks (CNN)**

## Overview

In this notebook, we will build a Convolutional Neural Network (CNN) to classify images of hand gestures: Rock, Paper, and Scissors. The process will include:

1. **Importing Libraries**
2. **Data Loading and Preparation**
3. **Model Building**
4. **Model Training**
5. **Model Evaluation**
6. **Making Predictions on New Images**

We will use TensorFlow and Keras for building the model and work through each step with detailed explanations.

### **1. Importing Necessary Libraries**

We start by importing essential libraries:

- **TensorFlow**: Main library for building and training the CNN.
- **TensorFlow Datasets**: For loading the Rock-Paper-Scissors dataset.
- **NumPy**: For numerical operations.
- **Matplotlib.pyplot**: For data visualization.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

### **2. Loading and Preparing the Dataset**

We will load the Rock-Paper-Scissors dataset from TensorFlow Datasets and preprocess it by:

- **Resizing Images**: Standardizing image sizes.
- **Normalizing Pixel Values**: Scaling pixel values to the range [0, 1].
- **Data Augmentation**: Enhancing the dataset with transformed images to improve model generalization.

# Resizing Images

In [ ]:
# def resize_images(folder_path, target_size):
#     for root, dirs, files in os.walk(folder_path):
#         for name in files:
#             if name.endswith(('.jpg', '.png', '.jpeg')):
#                 img_path = os.path.join(root, name)
#                 img = Image.open(img_path)
#                 img = img.resize(target_size)
#                 img.save(img_path)

# resize_images('dataset/train', (150, 150))
# resize_images('dataset/test', (150, 150))

# Combine 

In [ ]:
# # import tensorflow_datasets as tfds
# # import tensorflow as tf
# # import os
# # import numpy as np
# # from PIL import Image

# def download_and_prepare_tfds_rps():
#     """Download RPS dataset from TensorFlow Datasets"""
#     # Load the dataset
#     dataset, info = tfds.load('rock_paper_scissors', 
#                             split=['train', 'test'],
#                             with_info=True,
#                             as_supervised=True)
    
#     train_ds, test_ds = dataset
#     return train_ds, test_ds

# def save_tfds_images(dataset, output_path, split='train'):
#     """Save TensorFlow dataset images to disk"""
#     if not os.path.exists(output_path):
#         os.makedirs(output_path)
        
#     for image, label in dataset:
#         # Convert tensor to numpy array
#         image_array = image.numpy()
        
#         # Get class name based on label
#         class_names = ['rock', 'paper', 'scissors']
#         class_name = class_names[label]
        
#         # Create class directory if it doesn't exist
#         class_dir = os.path.join(output_path, split, class_name)
#         if not os.path.exists(class_dir):
#             os.makedirs(class_dir)
            
#         # Generate unique filename
#         filename = f"tfds_{split}_{class_name}_{len(os.listdir(class_dir))}.jpg"
        
#         # Resize image to match your current dataset size
#         image_pil = Image.fromarray(image_array)
#         image_pil = image_pil.resize((150, 150))
        
#         # Save image
#         image_pil.save(os.path.join(class_dir, filename))

# # Download and prepare TFDS dataset
# train_ds, test_ds = download_and_prepare_tfds_rps()

# # Save images to your dataset directory
# save_tfds_images(train_ds, './dataset', 'train')
# save_tfds_images(test_ds, './dataset', 'test')

# print("Dataset combination complete!")

In [ ]:
IMG_SIZE = (150, 150)
BATCH_SIZE = 8

# Load datasets using tf.keras.utils.image_dataset_from_directory
train_ds = tf.keras.utils.image_dataset_from_directory(
    'dataset/train',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'dataset/train',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    'dataset/test',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Define augmentation functions
def augment(image, label):
    # Random brightness
    image = tf.image.random_brightness(image, 0.2)
    
    # Random contrast
    image = tf.image.random_contrast(image, 0.8, 1.2)
    
    # Random flip
    image = tf.image.random_flip_left_right(image)
    
    # Random rotation
    # image = tf.image.rot90(
    #     image,
    #     tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    # )
    
    # Ensure pixel values are in [0,1]
    image = tf.clip_by_value(image, 0, 255)
    image = image / 255.0
    
    return image, label

# Configure datasets for performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.map(augment, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.cache()
train_ds = train_ds.shuffle(1000)
train_ds = train_ds.prefetch(AUTOTUNE)

# Normalize validation and test sets
val_ds = val_ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
val_ds = val_ds.cache()
val_ds = val_ds.prefetch(AUTOTUNE)

test_ds = test_ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
test_ds = test_ds.cache()
test_ds = test_ds.prefetch(AUTOTUNE)

### **Data Visualization**

Let's visualize a few samples from the training dataset to understand what the images look like.

In [ ]:
class_names = ['paper', 'rock', 'scissors']
# class_names = train_ds.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(6):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title(class_names[labels[i]])
        plt.axis('off')



### **4. Building the CNN Model**

We will build a Sequential CNN model consisting of:

- **Convolutional Layers**: To extract features from images.
- **Pooling Layers**: To reduce spatial dimensions.
- **Flatten Layer**: To convert the 2D feature maps into 1D feature vectors.
- **Dense Layers**: For classification based on extracted features.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (9, 9), activation='relu', input_shape=IMG_SIZE + (3,)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Conv2D(64, (9, 9), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

### **Compiling the Model**

We compile the model using:

- **Loss Function**: `sparse_categorical_crossentropy` for multi-class classification.
- **Optimizer**: `adam` optimizer for efficient training.
- **Metrics**: `accuracy` to evaluate the performance.

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

### **6. Training the Model**

We train the model using the training dataset for a certain number of epochs.

- **Epochs**: Number of times the model will cycle through the training data.

In [ ]:

# history = model.fit(
#     train_ds,
#     validation_data=test_ds,
#     epochs=10
# )

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

### **7. Evaluating the Model**

After training, we evaluate the model on the test dataset to check its performance on unseen data.

In [ ]:
test_loss, test_accuracy = model.evaluate(test_ds)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

### **Visualizing Training Results**

We plot the training and validation accuracy and loss to visualize the model's performance over epochs.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(5)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy over Epochs')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Loss over Epochs')
plt.show()

# Saving model

In [ ]:
model.save('./models/model-3.h5')

### **9. Making Predictions on New Images**

We can use the trained model to predict the class of new images.

- **Load and Preprocess the Image**: Resize and normalize the image.
- **Predict**: Use `model.predict` to get the prediction probabilities.
- **Interpret the Result**: Find the class with the highest probability.

In [ ]:
# First, define class names based on your dataset structure
class_names = ['paper', 'rock', 'scissors']  # Make sure this matches your folder structure order

def predict_image(image_path):
    """
    Predicts and displays the image with its prediction and confidence
    """
    # Load and preprocess the image
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=IMG_SIZE)
    image_array = tf.keras.preprocessing.image.img_to_array(image) / 255.0
    image_array = tf.expand_dims(image_array, 0)

    # Make prediction
    predictions = model.predict(image_array)
    predicted_class = np.argmax(predictions[0])
    confidence = np.max(predictions[0])

    # Display results
    plt.figure(figsize=(6, 6))
    plt.imshow(image_array[0])
    plt.title(f'Prediction: {class_names[predicted_class]} ({confidence * 100:.2f}%)')
    plt.axis('off')
    plt.show()

    # Print detailed probabilities
    for i, prob in enumerate(predictions[0]):
        print(f'{class_names[i]}: {prob*100:.2f}%')

def predict_image_simple(image_path):
    """
    Simpler version that just prints the prediction
    """
    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=IMG_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array = img_array / 255.0

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])
    confidence = np.max(predictions[0])
    
    print(f'Predicted class: {class_names[predicted_class]} (Confidence: {confidence*100:.2f}%)')

### **Testing the Model with a New Image**

Let's test the model with a new image of a hand gesture.

In [ ]:
# Provide the path to your image
image_path = './test-examples/scissors2.jpg'
predict_image(image_path)

# predict_image_simple(image_path)

version 1.0